In [49]:
from keras.layers import Input, Conv2D, GRU, Dense, Dropout, BatchNormalization, Bidirectional, Activation, Reshape
from keras.models import Model, load_model
import numpy as np
from preprocess import Augment
import time
import glob
import librosa
import librosa.display
import pyaudio
import sys
from queue import Queue
import matplotlib.pyplot as plt
from preprocess.td_utils import MaxSizeList

In [90]:
# model = load_model('/home/vietnv/PycharmProjects/trigger-word/model/model_6_0.9571788413098237.h5')
# model = load_model('/home/vietnv/PycharmProjects/trigger-word/model/model_mic2_0_0.9773299748110831.h5').
# model = load_model('/home/vietnv/PycharmProjects/trigger-word/model/model_mic2_0_0.9748110831234257.h5')
# model = load_model('/home/vietnv/PycharmProjects/trigger-word/model/model_mic2_0_0.9697732998982485.h5')
model = load_model('/home/vietnv/PycharmProjects/trigger-word/model/model_mic2_0_0.9798488664987406.h5')

In [91]:
def get_mel_single(path):
    aug = Augment.AudioAugmentation()
    data = aug.read_audio_file(path)
    S = librosa.feature.melspectrogram(data, sr=16000, n_mels=40, n_fft=400, hop_length=int(0.010 * 16000),
                                       power=1)
    x = librosa.power_to_db(S ** 2, ref=np.median)
    x = librosa.feature.mfcc(S=x, n_mfcc=40)
    x = x.swapaxes(0, 1)
    x = np.expand_dims(x, axis=2)

    return x


In [84]:
count = 0
for i in glob.glob('data_model2/train/error_pred/*'):
    data_mel = get_mel_single(i)
    detect = detect_triggerword_melspectrum(data_mel)
    if detect == 1:
        count += 1
        print(i)
#         shutil.move(i, 'data_model2/train/error_pred/' + str(time.time()) + '.wav')
print('count', count)

data_model2/train/error_pred/1563158335.6169171 (copy).wav
data_model2/train/error_pred/1563158239.5447574 (copy).wav
data_model2/train/error_pred/1563158232.4030988.wav
data_model2/train/error_pred/1563158355.6549108 (copy).wav
data_model2/train/error_pred/1563158214.811876 (copy).wav
data_model2/train/error_pred/1563158240.6384764 (copy).wav
data_model2/train/error_pred/1563158286.1853611.wav
data_model2/train/error_pred/1563158271.0320606 (copy).wav
data_model2/train/error_pred/1563158242.7315419 (copy).wav
data_model2/train/error_pred/1563158286.1853611 (copy).wav
data_model2/train/error_pred/1563158341.6488285.wav
data_model2/train/error_pred/1563158239.5447574.wav
data_model2/train/error_pred/1563158220.8894212.wav
data_model2/train/error_pred/1563158282.6073043 (copy).wav
data_model2/train/error_pred/1563158359.7324123.wav
data_model2/train/error_pred/1563158355.6549108.wav
data_model2/train/error_pred/1563158333.530038.wav
data_model2/train/error_pred/1563158291.2705317.wav
dat

data_model2/train/error_pred/1563158212.7002397.wav
data_model2/train/error_pred/1563158271.9147909 (copy).wav
data_model2/train/error_pred/1563158219.9186432.wav
data_model2/train/error_pred/1563158232.146747 (copy).wav
data_model2/train/error_pred/1563158253.7541227 (copy).wav
data_model2/train/error_pred/1563158336.3119314 (copy).wav
data_model2/train/error_pred/1563158346.5069158.wav
data_model2/train/error_pred/1563158326.5654056.wav
data_model2/train/error_pred/1563158276.8854814.wav
data_model2/train/error_pred/1563158277.9099731.wav
data_model2/train/error_pred/1563158342.6679993 (copy).wav
data_model2/train/error_pred/1563158237.1578062 (copy).wav
data_model2/train/error_pred/1563158361.5241263 (copy).wav
data_model2/train/error_pred/1563158255.1284819.wav
data_model2/train/error_pred/1563158288.1286392.wav
data_model2/train/error_pred/1563158282.6073043.wav
data_model2/train/error_pred/1563158342.9086432 (copy).wav
data_model2/train/error_pred/1563158219.9186432 (copy).wav
da

In [62]:
# for i in range(145):
#     data_mel = get_mel_single('from_mic_2_'+str(i)+'.wav')
#     print(i, detect_triggerword_melspectrum(data_mel))
# # data_mel = get_mel_single('/home/vietnv/PycharmProjects/trigger-word/data_model2/dev/happy/0b77ee66_nohash_1.wav')
# # print(detect_triggerword_melspectrum(data_mel))

In [99]:
aug = Augment.AudioAugmentation()
i = 0
def get_mel(data):
    global i
    data = data / 32767.0
    
    S = librosa.feature.melspectrogram(data, sr=16000, n_mels=40, n_fft=400, hop_length=int(0.010 * 16000),
                                       power=1)
    x = librosa.power_to_db(S ** 2, ref=np.median)
    x = librosa.feature.mfcc(S=x, n_mfcc=40)
    x = x.swapaxes(0, 1)
    x = np.expand_dims(x, axis=2)
    
    preds = detect_triggerword_melspectrum(x)
    if preds > 0:
        librosa.output.write_wav('create_data/non_happy/from_mic_2_' + str(i) + '.wav', data, sr=16000)
        i += 1
        
    return x

--...-..-.--.--..------

In [97]:
def detect_triggerword_melspectrum(x):
    x = np.expand_dims(x, axis=0)
    predictions = model.predict(x)
#     print('Prediction of Model: ', predictions)
    if predictions[0][1] > 0.5:
#         print('Prediction of Model: ', predictions, end='')
        return 1
    else:
        return 0

In [ ]:
check_kw = MaxSizeList(mx=5)
chunk_duration = 0.3 # Each read length in seconds from mic.
fs = 16000 # sampling rate for mic
chunk_samples = int(fs * chunk_duration) # Each read length in number of samples.
# Each model input data duration in seconds, need to be an integer numbers of chunk_duration
feed_duration = 1
feed_samples = int(fs * feed_duration)


### Audio stream
def get_audio_input_stream(callback):
    stream = pyaudio.PyAudio().open(
        format=pyaudio.paInt16,
        channels=1,
        rate=fs,
        input=True,
#         input_device_index=10,
        frames_per_buffer=chunk_samples,
        stream_callback=callback)
    return stream

# Queue to communiate between the audio callback and main thread
q = Queue()

run = True
silence_threshold = 200

# Run the demo for a timeout seconds
timeout = time.time() + 600  # 0.5 minutes from now

# Data buffer for the input wavform
data = np.zeros(feed_samples, dtype='int16')

def callback(in_data, frame_count, time_info, status):
    global run, timeout, data, silence_threshold    
    if time.time() > timeout:
        run = False        
    data0 = np.frombuffer(in_data, dtype='int16')
    if np.abs(data0).mean() < silence_threshold:
        sys.stdout.write('-')
        check_kw.push(0)
        return (in_data, pyaudio.paContinue)
    else:
        sys.stdout.write('.')
    data = np.append(data,data0)    
    if len(data) > feed_samples:
        data = data[-feed_samples:]
        q.put(data)
    return (in_data, pyaudio.paContinue)

stream = get_audio_input_stream(callback)
stream.start_stream()

try:
    while True:
        data = q.get()
        mel_spec = get_mel(data)
        preds = detect_triggerword_melspectrum(mel_spec)
        if preds > 0:
            sys.stdout.write('1')
#             check_kw.push(1)
#         else:
#             check_kw.push(0)
#         if sum(check_kw.get_list()) >= 3:
#             sys.stdout.write('1')
        
except (KeyboardInterrupt, SystemExit):
    stream.stop_stream()
    stream.close()
    timeout = time.time()
    run = False
        
stream.stop_stream()
stream.close()

---------....--.1.1----------..------------------....------..--------......1..------.....11.------.......-..----..--------..--..........-..1..1...----------------....------------..----..--....------------..----------------------------------------------..------------------------------------------------------------....------------------------------------------------..---.-.--..--------------------....----....-.-...----..----..----..--....11----------....------------....--------...-.-11----------------------------..------..--------..--..........-------.-.1..11--------....11--..--1--......--11--------------------------....11........--11-------.-.11..11----------..11..1----------------..--------------------------------------..........--......------......------..-------.-.------....-------.-...------..--------------------------------..--------------------..------....--........-.-...----------.....-.-....--------........------....11-------.-...----------....1--------....----------------------

In [71]:
from pydub import AudioSegment
import glob
import numpy as np


def random_choice_background():
    path = 'create_data/bg_youtube.wav'
    bg = AudioSegment.from_wav(path)
    rand_start_time = np.random.randint(0, len(bg) - 1000 - 1)
    return bg[rand_start_time: rand_start_time + 1000]

num_bg = 1000
while num_bg > 0:
    bg = random_choice_background()
    bg.set_frame_rate(16000).export('create_data/bg_youtube/'+'bg_' + str(num_bg) + '.wav', format='wav')
    num_bg -= 1
